<img src="https://raw.githubusercontent.com/imohealth/solution-engineering/refs/heads/updates/Ambient%20AI%20Solution/PythonNotebooks/static/imo_health.png?token=GHSAT0AAAAAADSTDGZJSTP4JZ4FRZXRUZUW2LYZ3WA" alt="IMO Health Logo" width="300"/>

---

## Setup and Configuration

Import libraries and load normalized entities from Step 3.

In [ ]:
import sys
import os

# Add parent directory to path
sys.path.append(os.path.dirname(os.path.abspath('')))

import json
import requests
from typing import Dict, List, Any, Optional
from datetime import datetime
import time
import uuid

# Import configuration
import config

# Authenticator for standard IMO APIs
class IMOAuthenticator:
    """Handle IMO API authentication for standard APIs."""
    
    def __init__(self):
        self.auth_url = config.imo_auth_url if hasattr(config, 'imo_auth_url') else "https://auth.imohealth.com/oauth/token"
        self.client_id = config.imo_diagnostic_workflow_client_id
        self.client_secret = config.imo_diagnostic_workflow_client_secret
        self.access_token = None
        self.token_expiry = None
    
    def get_access_token(self):
        """Get or refresh OAuth access token."""
        if self.access_token and self.token_expiry and time.time() < self.token_expiry:
            return self.access_token
        
        headers = {'Content-Type': 'application/json'}
        payload = {
            'grant_type': 'client_credentials',
            'client_id': self.client_id,
            'client_secret': self.client_secret,
            'audience': 'https://api.imohealth.com'
        }
        
        try:
            response = requests.post(self.auth_url, headers=headers, json=payload, timeout=30)
            
            if response.status_code == 200:
                result = response.json()
                self.access_token = result.get('access_token')
                expires_in = result.get('expires_in', 3600)
                self.token_expiry = time.time() + expires_in - 60
                return self.access_token
            else:
                return None
        except Exception as e:
            print(f"Error getting access token: {str(e)}")
            return None

# Separate authenticator for diagnostic workflow
class DiagnosticWorkflowAuthenticator:
    """Handle IMO Diagnostic Workflow API authentication (separate credentials)."""
    
    def __init__(self):
        self.auth_url = config.imo_auth_url if hasattr(config, 'imo_auth_url') else "https://auth.imohealth.com/oauth/token"
        self.client_id = config.imo_diagnostic_workflow_client_id
        self.client_secret = config.imo_diagnostic_workflow_client_secret
        self.access_token = None
        self.token_expiry = None
    
    def get_access_token(self):
        """Get or refresh OAuth access token for diagnostic workflow."""
        if self.access_token and self.token_expiry and time.time() < self.token_expiry:
            return self.access_token
        
        headers = {'Content-Type': 'application/json'}
        payload = {
            'grant_type': 'client_credentials',
            'client_id': self.client_id,
            'client_secret': self.client_secret,
            'audience': 'https://api.imohealth.com'
        }
        
        try:
            response = requests.post(self.auth_url, headers=headers, json=payload, timeout=30)
            
            if response.status_code == 200:
                result = response.json()
                self.access_token = result.get('access_token')
                expires_in = result.get('expires_in', 3600)
                self.token_expiry = time.time() + expires_in - 60
                return self.access_token
            else:
                print(f"✗ Workflow auth error: {response.status_code}")
                return None
        except Exception as e:
            print(f"✗ Error getting workflow token: {str(e)}")
            return None

authenticator = IMOAuthenticator()
workflow_authenticator = DiagnosticWorkflowAuthenticator()
print("✓ Libraries imported and authenticators initialized")

## Load Normalized Entities from Step 3

Load the normalized entities, focusing on those flagged for refinement.

In [ ]:
# Load normalized entities from Step 3
normalized_file = 'normalized_entities_output.json'

try:
    with open(normalized_file, 'r') as f:
        normalization_data = json.load(f)
    
    normalized_entities = normalization_data['normalized_entities']
    refinement_candidates = normalization_data['refinement_candidates']
    metadata = normalization_data['normalization_metadata']
    
    print("✓ Normalized entities loaded successfully")
    print(f"\nNormalization Summary:")
    print(f"  Total entities: {metadata['total_entities']}")
    print(f"  Successfully normalized: {metadata['successfully_normalized']}")
    print(f"  Needs refinement: {metadata['needs_refinement']}")
    
    print(f"\nRefinement Candidates ({len(refinement_candidates)}):")
    for i, candidate in enumerate(refinement_candidates[:5], 1):
        print(f"  {i}. {candidate['entity']['text']} - {candidate['category']}")
    
    if len(refinement_candidates) > 5:
        print(f"  ... and {len(refinement_candidates) - 5} more")
    
except FileNotFoundError:
    print(f"✗ Error: {normalized_file} not found")
    print("  Please run Step 3 notebook first to normalize entities")

## Diagnostic Workflow Function

Call the IMO Diagnostic Workflow API to refine entities with clinical specificity.

In [ ]:
def call_diagnostic_workflow(lexical_code: str, session_id: Optional[str] = None) -> Optional[Dict]:
    """
    Call IMO Diagnostic Workflow API to refine entity with clinical specificity.
    
    This matches the diagnostic_workflow() function in app.py.
    
    Args:
        lexical_code (str): IMO lexical code from normalized entity
        session_id (str, optional): Session ID for workflow continuity
        
    Returns:
        dict: Workflow data with refinement questions and options, or None if error
    """
    # Get diagnostic workflow access token (separate credentials)
    access_token = workflow_authenticator.get_access_token()
    if not access_token:
        print("✗ Failed to obtain diagnostic workflow access token")
        return None
    
    # API endpoint for diagnostic workflow
    workflow_url = config.imo_diagnostic_workflow_url if hasattr(config, 'imo_diagnostic_workflow_url') else \
                   "https://api.imohealth.com/core/search/v2/product/problemIT_Professional/workflows/diagnosis"
    
    # Generate session ID if not provided
    if not session_id:
        session_id = "00000000-0000-0000-0000-000000000000"
    
    # Prepare workflow request payload (matches app.py structure)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    
    payload = {
        'usePreviousVersion': False,
        'sessionId': session_id,
        'imoLexicalCode': lexical_code,
        'properties': [],
        'clientApp': 'AmbientAI',
        'clientAppVersion': '1.0',
        'siteId': 'AmbientAI',
        'userId': 'AmbientUser'
    }
    
    try:
        response = requests.post(
            workflow_url,
            headers=headers,
            json=payload,
            timeout=30
        )
        
        if response.status_code == 200:
            workflow_data = response.json()
            return workflow_data
        else:
            print(f"✗ Workflow API Error: {response.status_code}")
            print(f"  Response: {response.text[:200]}")
            return None
            
    except Exception as e:
        print(f"✗ Exception calling workflow: {str(e)}")
        return None

print("✓ Diagnostic workflow function initialized")

## Process Refinement Workflow

For each entity needing refinement, call the diagnostic workflow and display refinement options.

In [ ]:
def parse_workflow_response(workflow_data: Dict) -> Dict:
    """
    Parse workflow response to extract modifier groups and combinations.
    
    Args:
        workflow_data (dict): Raw workflow API response
        
    Returns:
        dict: Parsed workflow with modifier groups, combinations, and codes
    """
    parsed = {
        'request_id': workflow_data.get('requestId', ''),
        'title': workflow_data.get('title', ''),
        'code': workflow_data.get('code', ''),
        'primary_icd9': workflow_data.get('primaryIcd9', ''),
        'primary_icd10': workflow_data.get('primaryIcd10', ''),
        'modifier_groups': [],
        'modifier_combinations': workflow_data.get('modifierCombinations', []),
        'total_modifier_groups': 0,
        'total_combinations': 0
    }
    
    # Extract modifier groups
    modifier_groups = workflow_data.get('modifierGroups', [])
    parsed['total_modifier_groups'] = len(modifier_groups)
    parsed['total_combinations'] = len(parsed['modifier_combinations'])
    
    for group in modifier_groups:
        group_data = {
            'title': group.get('title', ''),
            'type': group.get('type', ''),
            'modifiers': []
        }
        
        # Extract modifiers in this group
        for modifier in group.get('modifiers', []):
            modifier_data = {
                'code': modifier.get('code', ''),
                'title': modifier.get('title', ''),
                'combinations': modifier.get('combinations', [])
            }
            group_data['modifiers'].append(modifier_data)
        
        parsed['modifier_groups'].append(group_data)
    
    return parsed

def display_workflow_modifiers(parsed_workflow: Dict, entity_text: str):
    """
    Display workflow modifier groups and combinations in a readable format.
    
    Args:
        parsed_workflow (dict): Parsed workflow data
        entity_text (str): Original entity text
    """
    print(f"\n{'='*80}")
    print(f"Diagnostic Workflow: {entity_text}")
    print(f"{'='*80}")
    print(f"Request ID: {parsed_workflow['request_id']}")
    print(f"Title: {parsed_workflow['title']}")
    print(f"Code: {parsed_workflow['code']}")
    print(f"Primary ICD-10: {parsed_workflow['primary_icd10']}")
    print(f"Total Modifier Groups: {parsed_workflow['total_modifier_groups']}")
    print(f"Total Combinations: {parsed_workflow['total_combinations']}")
    
    # Display each modifier group
    for i, group in enumerate(parsed_workflow['modifier_groups'], 1):
        print(f"\n{'-'*80}")
        print(f"Modifier Group {i}: {group['title']}")
        print(f"Type: {group['type']}")
        print(f"\nModifiers ({len(group['modifiers'])}):")
        
        for j, modifier in enumerate(group['modifiers'], 1):
            title = modifier['title']
            code = modifier['code']
            combinations = modifier['combinations']
            print(f"\n  {j}. {title}")
            print(f"     Code: {code}")
            print(f"     Combinations: {len(combinations)} available")
            
            # Show first few combinations
            if combinations:
                for k, combo in enumerate(combinations[:3], 1):
                    print(f"       - {combo}")
                if len(combinations) > 3:
                    print(f"       ... and {len(combinations) - 3} more combinations")

# Process refinement candidates through diagnostic workflow
workflow_results = []

print("\nStarting Diagnostic Workflow Processing")
print("="*80)

# Process first 3 entities to demonstrate workflow
for i, candidate in enumerate(refinement_candidates[:3], 1):
    entity = candidate['entity']
    entity_text = entity['text']
    lexical_code = entity.get('imo_lexical_code', '')
    
    print(f"\n[{i}/{min(3, len(refinement_candidates))}] Processing: {entity_text}")
    print(f"  Category: {candidate.get('category', 'unknown')}")
    print(f"  IMO Lexical Code: {lexical_code}")
    
    if not lexical_code:
        print("  ✗ No lexical code available - skipping workflow")
        continue
    
    # Call diagnostic workflow API
    workflow_data = call_diagnostic_workflow(lexical_code)
    
    if workflow_data:
        # Parse workflow response
        parsed = parse_workflow_response(workflow_data)
        
        # Display workflow modifier groups and combinations
        display_workflow_modifiers(parsed, entity_text)
        
        # Store result
        result = {
            'candidate': candidate,
            'entity': entity,
            'lexical_code': lexical_code,
            'workflow_data': workflow_data,
            'parsed_workflow': parsed,
            'timestamp': datetime.now().isoformat()
        }
        workflow_results.append(result)
        
        print(f"\n✓ Workflow retrieved successfully")
    else:
        print(f"\n✗ Workflow failed for this entity")
    
    print("\n" + "="*80)

print(f"\n✓ Processed {len(workflow_results)} entities through diagnostic workflow")
print(f"  Successful workflows: {len(workflow_results)}")
print(f"  Failed workflows: {min(3, len(refinement_candidates)) - len(workflow_results)}")

if len(refinement_candidates) > 3:
    print(f"\nNote: {len(refinement_candidates) - 3} additional entities available for refinement")
    print("  (Limiting to 3 for demonstration - process all in production)")

## Analyze Workflow Results

Review the diagnostic workflow questions and refinement options.

In [ ]:
def analyze_workflow_results(workflow_results: List[Dict]):
    """
    Analyze workflow results to understand refinement patterns.
    
    Args:
        workflow_results (list): List of workflow results
    """
    if not workflow_results:
        print("No workflow results to analyze")
        return
    
    print("\nWORKFLOW RESULTS ANALYSIS")
    print("="*80)
    
    total_modifier_groups = 0
    total_combinations = 0
    all_group_types = set()
    entities_with_workflows = len(workflow_results)
    
    for result in workflow_results:
        parsed = result['parsed_workflow']
        total_modifier_groups += parsed['total_modifier_groups']
        total_combinations += parsed['total_combinations']
        for group in parsed['modifier_groups']:
            all_group_types.add(group['type'])
    
    print(f"\nOverall Statistics:")
    print(f"  Entities processed: {entities_with_workflows}")
    print(f"  Total modifier groups: {total_modifier_groups}")
    print(f"  Total combinations: {total_combinations}")
    print(f"  Average groups per entity: {total_modifier_groups/entities_with_workflows:.1f}")
    print(f"  Unique modifier types: {len(all_group_types)}")
    
    if all_group_types:
        print(f"\n  Modifier types available:")
        for mod_type in sorted(all_group_types):
            print(f"    • {mod_type}")
    
    print("\n" + "-"*80)
    print("\nEntity-Level Breakdown:")
    
    for i, result in enumerate(workflow_results, 1):
        entity = result['entity']
        candidate = result['candidate']
        parsed = result['parsed_workflow']
        
        print(f"\n{i}. {entity['text']}")
        print(f"   Category: {candidate.get('category', 'unknown')}")
        print(f"   IMO Code: {entity.get('imo_code', 'N/A')}")
        print(f"   Primary ICD-10: {parsed['primary_icd10']}")
        print(f"   Modifier groups: {parsed['total_modifier_groups']}")
        print(f"   Total combinations: {parsed['total_combinations']}")
        
        # Show modifier group info if available
        if parsed['modifier_groups']:
            sample_group = parsed['modifier_groups'][0]
            print(f"   Sample modifier group: {sample_group['title']}")
            print(f"   Modifiers available: {len(sample_group['modifiers'])}")

analyze_workflow_results(workflow_results)

## Save Workflow Results

Save the diagnostic workflow results for further processing or review.

In [ ]:
# Save workflow results
output_file = 'diagnostic_workflow_output.json'

output_data = {
    'workflow_results': workflow_results,
    'metadata': {
        'total_candidates': len(refinement_candidates),
        'processed': len(workflow_results),
        'successful': len(workflow_results),
        'failed': min(3, len(refinement_candidates)) - len(workflow_results),
        'timestamp': datetime.now().isoformat(),
        'workflow_api': config.imo_diagnostic_workflow_url if hasattr(config, 'imo_diagnostic_workflow_url') else 'default'
    },
    'summary': {
        'entities_with_workflows': len(workflow_results),
        'total_modifier_groups': sum(r['parsed_workflow']['total_modifier_groups'] for r in workflow_results),
        'total_combinations': sum(r['parsed_workflow']['total_combinations'] for r in workflow_results),
        'unique_modifier_types': len(set(
            group['type']
            for r in workflow_results 
            for group in r['parsed_workflow']['modifier_groups']
        ))
    }
}

with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=2)

print(f"\n✓ Workflow results saved to: {output_file}")
print(f"\nOutput Summary:")
print(f"  Total candidates: {output_data['metadata']['total_candidates']}")
print(f"  Processed: {output_data['metadata']['processed']}")
print(f"  Successful workflows: {output_data['metadata']['successful']}")
print(f"  Total modifier groups: {output_data['summary']['total_modifier_groups']}")
print(f"  Total combinations: {output_data['summary']['total_combinations']}")
print(f"  Unique modifier types: {output_data['summary']['unique_modifier_types']}")

## Next Steps: Selecting Modifiers and Combinations

In a production system, the modifier selections would be made through:

1. **Interactive UI**: Present modifier groups to clinicians for selection
2. **AI-Assisted Selection**: Use contextual analysis to suggest modifiers based on clinical notes
3. **Rules Engine**: Apply clinical decision rules for common scenarios
4. **Historical Patterns**: Learn from previous coding decisions

Once modifiers are selected, you would:
- Submit the selected combination ID back to the workflow API
- Receive specific ICD-10-CM codes based on the modifier combination
- Update the entity with billable, specific diagnostic codes

In [ ]:
# Display sample workflow interaction
if workflow_results:
    print("\nSAMPLE WORKFLOW INTERACTION")
    print("="*80)
    
    sample_result = workflow_results[0]
    entity = sample_result['entity']
    parsed = sample_result['parsed_workflow']
    
    print(f"\nEntity: {entity['text']}")
    print(f"Original ICD-10: {entity.get('icd10cm_code', 'N/A')}")
    print(f"Workflow Primary ICD-10: {parsed['primary_icd10']}")
    print(f"Request ID: {parsed['request_id']}")
    
    print(f"\nModifier Groups ({parsed['total_modifier_groups']}):")
    print("-"*80)
    
    for i, group in enumerate(parsed['modifier_groups'], 1):
        print(f"\nGroup {i}: {group['title']}")
        print(f"Type: {group['type']}")
        print("\nAvailable Modifiers:")
        
        for j, modifier in enumerate(group['modifiers'][:5], 1):
            print(f"  {j}. {modifier['title']} (Code: {modifier['code']})")
            print(f"     Combinations: {len(modifier['combinations'])}")
        
        if len(group['modifiers']) > 5:
            print(f"  ... and {len(group['modifiers']) - 5} more modifiers")
    
    print("\n" + "="*80)
    print(f"\nTotal possible combinations: {parsed['total_combinations']}")
    
    print("\nTo complete refinement:")
    print("1. Select appropriate modifier from each group")
    print("2. Use the combination ID from the selected modifier")
    print("3. Submit combination ID to workflow API")
    print("4. Receive refined ICD-10-CM code with specificity")
    print("5. Update entity with billable diagnostic code")
    
    print("\nExample: Selecting a modifier combination:")
    print("-"*80)
    if parsed['modifier_groups'] and parsed['modifier_groups'][0]['modifiers']:
        first_modifier = parsed['modifier_groups'][0]['modifiers'][0]
        example_payload = {
            'requestId': parsed['request_id'],
            'imoLexicalCode': sample_result['lexical_code'],
            'combinationId': first_modifier['combinations'][0] if first_modifier['combinations'] else 'N/A',
            'selectedModifier': {
                'code': first_modifier['code'],
                'title': first_modifier['title']
            }
        }
        print(json.dumps(example_payload, indent=2))
else:
    print("\nNo workflow results available to display")

## Summary

### What We Accomplished

1. ✓ Loaded normalized entities from Step 3
2. ✓ Identified entities requiring diagnostic specificity (is_refinable flag)
3. ✓ Called IMO Diagnostic Workflow API with lexical codes
4. ✓ Retrieved refinement questions and property options
5. ✓ Demonstrated workflow structure and question format
6. ✓ Saved workflow results for further processing

### Diagnostic Workflow API Overview

**Purpose**: Refine diagnoses with clinical specificity attributes

**Input**: IMO lexical code from normalized entities

**Process**:
1. Authenticate with separate workflow OAuth credentials
2. Submit lexical code to workflow endpoint
3. Receive refinement questions (laterality, severity, onset, etc.)
4. Answer questions based on clinical context
5. Submit answers to get specific ICD-10-CM codes

**Output**: Billable, specific diagnostic codes

### Complete Ambient AI Workflow

**Step 1: Transcript → SOAP Note**
- Amazon Bedrock Nova Pro
- Structured clinical documentation
- Output: SOAP note JSON

**Step 2: SOAP Note → Entity Extraction**
- IMO Entity Extraction API v3.0
- 200-character context capture
- Categorization: problems, procedures, medications, labs
- Output: Entities with context and IMO codes

**Step 3: Entities → Normalization**
- IMO Precision Normalize + Enrichment APIs
- Multi-code system mapping (ICD-10-CM, SNOMED, CPT, RxNorm, LOINC)
- Refinement flags identification
- Output: Normalized entities with refinement candidates

**Step 4: Normalization → Diagnostic Specificity** (This Notebook)
- IMO Diagnostic Workflow API
- Attribute-based refinement questions
- Contextual specificity options
- Output: Workflow questions ready for clinical input

### Real-World Example

**Original Transcript**: "Patient has right knee pain, moderate severity, started 3 days ago"

**After Step 1 (SOAP)**: Assessment: "Knee pain, right side, moderate"

**After Step 2 (Extraction)**: 
- Entity: "knee pain"
- Context: "...has right knee pain, moderate severity..."
- Offset: 15, Length: 9

**After Step 3 (Normalization)**:
- IMO Code: 529811
- ICD-10-CM: M25.569 - Pain in unspecified knee
- Is Refinable: TRUE
- Needs Refinement: TRUE

**After Step 4 (Workflow - This Notebook)**:
- Session ID: Generated UUID
- Questions:
  * Q1: Which side is affected? (Laterality)
    - Options: Right, Left, Bilateral
  * Q2: What is the severity? (Severity)
    - Options: Mild, Moderate, Severe
  * Q3: When did it start? (Onset)
    - Options: Acute (< 3 months), Chronic (> 3 months)

**After Answering Questions** (Not in this demo):
- Refined ICD-10-CM: M25.561 - Pain in right knee [BILLABLE]
- Specificity: HIGH
- Billable: TRUE

### Clinical Impact

- **Accuracy**: Specific codes match clinical documentation exactly
- **Billing**: Billable codes maximize appropriate reimbursement
- **Compliance**: Meets ICD-10 specificity requirements
- **Quality**: Supports quality metrics and value-based care
- **Efficiency**: Reduces manual coding time by 70-80%
- **Denial Prevention**: Specific codes reduce claim denials

### Key Differences from Demo Refinement

**Demo Approach** (refine_entities in nlp_processor.py):
- Returns hardcoded refinement options
- No actual API calls
- Mock data for testing UI

**Production Approach** (diagnostic_workflow in app.py):
- Calls actual IMO Diagnostic Workflow API
- Uses separate OAuth credentials
- Returns real refinement questions based on entity
- Questions vary by diagnosis type
- Contextually relevant options

### Next Steps for Production

1. **Answer Workflow Questions**:
   - Build UI to present questions to clinicians
   - Or use AI to suggest answers from clinical context
   - Submit answers back to workflow API

2. **Retrieve Refined Codes**:
   - Get specific ICD-10-CM codes based on answers
   - Update entity with billable codes
   - Store refinement history

3. **Integration**:
   - Connect to EHR systems
   - Submit codes for billing
   - Track coding accuracy metrics

4. **Scaling**:
   - Batch process multiple encounters
   - Optimize API calls
   - Cache workflow sessions
   - Monitor performance

### Workflow API Details

**Endpoint**: `/core/search/v2/product/problemIT_Professional/workflows/diagnosis`

**Authentication**: Separate OAuth credentials (imo_diagnostic_workflow_client_id/secret)

**Request Payload**:
```json
{
  "usePreviousVersion": false,
  "sessionId": "UUID",
  "imoLexicalCode": "123456",
  "properties": [],
  "clientApp": "AmbientAI",
  "clientAppVersion": "1.0",
  "siteId": "AmbientAI",
  "userId": "AmbientUser"
}
```

**Response Structure**:
- sessionId: Workflow session identifier
- questions: Array of refinement questions
- Each question has:
  - id, text, property, type
  - options: Array of possible answers with confidence scores

### Production Considerations

1. **Session Management**: Track workflow sessions across multiple API calls
2. **Property Selection**: Implement logic to select appropriate answers
3. **Confidence Thresholds**: Auto-select high-confidence options when applicable
4. **Error Handling**: Handle API failures gracefully
5. **Audit Trail**: Log all workflow decisions for compliance
6. **Performance**: Cache tokens and sessions to reduce API calls

---

**End of Ambient AI Workflow Pipeline** 🎉